In [2]:
import pandas as pd

# 1. Load both files
master_df = pd.read_csv('master.csv')
sched_df = pd.read_excel('Irrigation + ALL Elemental Fractions schedule for one plant (100N).xlsx')

# 2. Fix the dates
master_df['timestamp'] = pd.to_datetime(master_df['timestamp'])

sched_df['DATE'] = sched_df['DATE'].astype(str).str.split(' ').str[0] 
sched_df['TIME'] = sched_df['TIME'].astype(str)
sched_df['datetime'] = pd.to_datetime(sched_df['DATE'] + ' ' + sched_df['TIME'])

# FIX: Snap the schedule's weird minutes to the nearest 10-minute interval
sched_df['datetime_rounded'] = sched_df['datetime'].dt.round('10min')

# 3. List the chemical columns to update
chem_cols = [
    'Ammonium Nitrate [mg] -NH4NO3', 
    'Monopotassium Phosphate[mg] -KH2PO4', 
    'Potassium Chloride[mg] - KCL', 
    'Phosphoric acid[mg]-H3PO4', 
    'Kortin [mg]', 
    'Gypsum - CaSO4*2H2O [mg]'
]

# 4. Zero out the broken chemical columns in the master file
for col in chem_cols:
    master_df[col] = 0.0

# 5. Bring in the exact, correct values from the schedule!
# Group by the *rounded* datetime and sum the chemicals
sched_df_grouped = sched_df.groupby('datetime_rounded')[chem_cols].sum()

master_df = master_df.set_index('timestamp')
master_df.update(sched_df_grouped)

# 6. Save the perfectly synced file
master_df.reset_index().to_csv('master_fixed_final.csv', index=False)
print("Saved perfectly synced file as 'master_fixed_final.csv'")

Saved perfectly synced file as 'master_fixed_final.csv'
